In [1]:
import os.path
import tensorflow as tf
import helper
import warnings
from distutils.version import LooseVersion
import project_tests as tests

In [2]:
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer.  You are using {}'.format(tf.__version__)
print('TensorFlow Version: {}'.format(tf.__version__))

TensorFlow Version: 1.4.0


In [3]:
# download data
data_dir = './data'
helper.maybe_download_pretrained_vgg(data_dir)

In [4]:
tests.test_for_kitti_dataset(data_dir)

Tests Passed


In [5]:
def load_vgg(sess, vgg_path):
    """
    Load Pretrained VGG Model into TensorFlow.
    :param sess: TensorFlow Session
    :param vgg_path: Path to vgg folder, containing "variables/" and "saved_model.pb"
    :return: Tuple of Tensors from VGG model (image_input, keep_prob, layer3_out, layer4_out, layer7_out)
    """
    # TODO: Implement function
    #   Use tf.saved_model.loader.load to load the model and weights
    vgg_tag = 'vgg16'
    vgg_input_tensor_name = 'image_input:0'
    vgg_keep_prob_tensor_name = 'keep_prob:0'
    vgg_layer3_out_tensor_name = 'layer3_out:0'
    vgg_layer4_out_tensor_name = 'layer4_out:0'
    vgg_layer7_out_tensor_name = 'layer7_out:0'
    
    tf.saved_model.loader.load(sess, [vgg_tag], vgg_path)
    graph = tf.get_default_graph()
    image_input = graph.get_tensor_by_name(vgg_input_tensor_name)
    keep_prob = graph.get_tensor_by_name(vgg_keep_prob_tensor_name)
    layer3_out = graph.get_tensor_by_name(vgg_layer3_out_tensor_name)
    layer4_out = graph.get_tensor_by_name(vgg_layer4_out_tensor_name)
    layer7_out = graph.get_tensor_by_name(vgg_layer7_out_tensor_name)
    
    return image_input, keep_prob, layer3_out, layer4_out, layer7_out
tests.test_load_vgg(load_vgg, tf)

Tests Passed


In [6]:
def layers(vgg_layer3_out, vgg_layer4_out, vgg_layer7_out, num_classes):
    """
    Create the layers for a fully convolutional network.  Build skip-layers using the vgg layers.
    :param vgg_layer3_out: TF Tensor for VGG Layer 3 output
    :param vgg_layer4_out: TF Tensor for VGG Layer 4 output
    :param vgg_layer7_out: TF Tensor for VGG Layer 7 output
    :param num_classes: Number of classes to classify
    :return: The Tensor for the last layer of output
    """
    # TODO: Implement function
    l2_regularizer = tf.contrib.layers.l2_regularizer(1E-3)
    conv_1x1_layer7 = tf.layers.conv2d(vgg_layer7_out, num_classes, 1, 1, padding="same", kernel_regularizer=l2_regularizer)
    upsample_layer7 = tf.layers.conv2d_transpose(conv_1x1_layer7, num_classes, 4, (2, 2), padding="same", kernel_regularizer=l2_regularizer)

    layer4_out_scaled = tf.multiply(vgg_layer4_out, 0.01, name='layer4_out_scaled')
    conv_1x1_layer4 = tf.layers.conv2d(layer4_out_scaled, num_classes, 1, 1, padding="same", kernel_regularizer=l2_regularizer)
    blended_layer4 = tf.add(conv_1x1_layer4, upsample_layer7)
    upsample_layer4 = tf.layers.conv2d_transpose(blended_layer4, num_classes, 4, (2, 2), padding="same", kernel_regularizer=l2_regularizer)

    layer3_out_scaled = tf.multiply(vgg_layer3_out, 0.0001, name='layer3_out_scaled')
    conv_1x1_layer3 = tf.layers.conv2d(vgg_layer3_out, num_classes, 1, 1, padding="same", kernel_regularizer=l2_regularizer)
    blended_layer3 = tf.add(conv_1x1_layer3, upsample_layer4)
    upsample_layer3 = tf.layers.conv2d_transpose(blended_layer3, num_classes, 16, (8, 8), padding="same", kernel_regularizer=l2_regularizer)

    # tf.Print(output, [tf.shape(output)])
    return upsample_layer3
tests.test_layers(layers)

Tests Passed


In [7]:
def optimize(nn_last_layer, correct_label, learning_rate, num_classes):
    """
    Build the TensorFLow loss and optimizer operations.
    :param nn_last_layer: TF Tensor of the last layer in the neural network
    :param correct_label: TF Placeholder for the correct label image
    :param learning_rate: TF Placeholder for the learning rate
    :param num_classes: Number of classes to classify
    :return: Tuple of (logits, train_op, cross_entropy_loss)
    """
    # TODO: Implement function
    logits = tf.reshape(nn_last_layer, (-1, num_classes))            # no need to reshape actually
    labels = tf.reshape(correct_label, (-1, num_classes))
    cross_entropy_loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
                                        logits=nn_last_layer, labels=correct_label))
    # IMPORTANT: need to manually add regularization loss to total loss
    reg_losses = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
    total_loss = tf.add_n([cross_entropy_loss] + reg_losses)
    optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate)
    train_op = optimizer.minimize(total_loss)
    return logits, train_op, total_loss
tests.test_optimize(optimize)

Tests Passed


In [31]:
import time
def train_nn(sess, epochs, batch_size, get_batches_fn, train_op, cross_entropy_loss, input_image,
             correct_label, keep_prob, learning_rate):
    """
    Train neural network and print out the loss during training.
    :param sess: TF Session
    :param epochs: Number of epochs
    :param batch_size: Batch size
    :param get_batches_fn: Function to get batches of training data.  Call using get_batches_fn(batch_size)
    :param train_op: TF Operation to train the neural network
    :param cross_entropy_loss: TF Tensor for the amount of loss
    :param input_image: TF Placeholder for input images
    :param correct_label: TF Placeholder for label images
    :param keep_prob: TF Placeholder for dropout keep probability
    :param learning_rate: TF Placeholder for learning rate
    """
    # TODO: Implement function
    for epoch in range(epochs):
        print("****************** Starting Epoch {} *******************".format(epoch+1))
        t0 = time.time()
        for images, labels in get_batches_fn(batch_size):
            _, loss = sess.run([train_op, cross_entropy_loss],
                               feed_dict ={input_image: images, correct_label: labels,
                                           keep_prob: 0.5, learning_rate: 0.001})
            print("Current loss is {}".format(loss))
            ti = time.time()
            print("{}s has past for this epoch".format(ti - t0))
        print("Total loss after epoch {} is {}".format(epoch+1, loss))

tests.test_train_nn(train_nn)

In [24]:
restart = False
train_stage = False
num_classes = 2
image_shape = (160, 576)
data_dir = './data'
runs_dir = './runs'
tests.test_for_kitti_dataset(data_dir)

# Download pretrained vgg model
helper.maybe_download_pretrained_vgg(data_dir)

ckpt_prefix = './tmp/'

# OPTIONAL: Train and Inference on the cityscapes dataset instead of the Kitti dataset.
# You'll need a GPU with at least 10 teraFLOPS to train on.
#  https://www.cityscapes-dataset.com/

# Path to vgg model
vgg_path = os.path.join(data_dir, 'vgg')
# Create function to get batches
get_batches_fn = helper.gen_batch_function(os.path.join(data_dir, 'data_road/training'), image_shape)

# OPTIONAL: Augment Images for better results
#  https://datascience.stackexchange.com/questions/5224/how-to-prepare-augment-images-for-neural-network


In [30]:
with tf.Session() as sess:
    # Build NN using load_vgg, layers, and optimize function
    image_input, keep_prob, layer3_out, layer4_out, layer7_out = load_vgg(sess, vgg_path)
    output_layer = layers(layer3_out, layer4_out, layer7_out, num_classes)
    correct_label = tf.placeholder(tf.float32, [None, None, None, num_classes])
    learning_rate = tf.placeholder(tf.float32)
    logits, train_op, total_loss = optimize(output_layer, correct_label, learning_rate, num_classes)
    mean_iou, iou_update_op = tf.metrics.mean_iou(tf.reshape(correct_label, (-1, num_classes)), tf.nn.softmax(logits), num_classes)
    print("NN constructed!")

    # Train NN using the train_nn function
    sess.run(tf.global_variables_initializer())
    sess.run(tf.local_variables_initializer())
    
    saver = tf.train.Saver(max_to_keep = 5, keep_checkpoint_every_n_hours = 3)
    latest_checkpoint = tf.train.latest_checkpoint(ckpt_prefix)
    if latest_checkpoint and not restart:
        print("Restoring from {}".format(latest_checkpoint))
        saver.restore(sess, latest_checkpoint)
        print("Session Restored!")
    n_epoch = 10
    batch_size = 20
    if train_stage:
        train_nn(sess, n_epoch, batch_size, get_batches_fn, train_op, total_loss, image_input,
            correct_label, keep_prob, learning_rate)
        # Save trained NN to continue next time
        saver.save(sess, ckpt_prefix + 'sess.ckpt')
        print("Session Saved!")

    # Calculate IOU
    for images, labels in get_batches_fn(2):
        sess.run(iou_update_op, feed_dict={image_input: images, correct_label: labels,
                                           keep_prob: 1.0,})
        break
    iou_score = sess.run(mean_iou)
    print("Mean IOU score is: {}".format(iou_score))

In [23]:
tf.train.latest_checkpoint('./tmp/')

'./tmp/sess.ckpt'

In [29]:
tf.reset_default_graph()

In [ ]:
from moviepy.editor import VideoFileClip
import scipy
clip = VideoFileClip('driving.mp4').subclip(0,1)
    
def process_video(sess, logits, keep_prob, image_pl, image_shape, clip):
    def image_pipeline(img):
        image = scipy.misc.imresize(img, image_shape)
        im_softmax = sess.run(
                [tf.nn.softmax(logits)],
                {keep_prob: 1.0, image_pl: [image]})
        im_softmax = im_softmax[0][:, 1].reshape(image_shape[0], image_shape[1])
        segmentation = (im_softmax > 0.5).reshape(image_shape[0], image_shape[1], 1)
        mask = np.dot(segmentation, np.array([[0, 255, 0, 127]]))
        mask = scipy.misc.toimage(mask, mode="RGBA")
        street_im = scipy.misc.toimage(image)
        street_im.paste(mask, box=None, mask=mask)
        return np.array(street_im)
    new_clip = clip.fl_image(image_pipeline)
    new_clip.write_videofile('result.mp4')